# Inflation vs. Wages (Overall)
2025-09-14

## 1. Import Functions and Set Variables

In [35]:
from _notebook_setup import *
import plotly.express as px
import plotly.graph_objects as go

hit_api = True
save_output = False

current_year= int(datetime.now().strftime("%Y"))
years = list(range(1984, current_year + 1)) # starting in 1983 instead of 1979 because CPI data is normalized to 8/1983 due to updated expenditure weights

## 2. Import Data

### 2.a CPI (Unadjusted)

In [36]:
# got series_ids from https://www.bls.gov/help/hlpforma.htm#OCWC
base_series_ids = {
    "CUUR": "CPI (Unadjusted)"
 }

regions = {
    "0000": "National",
}

items = {
    "SA0": "All items",
    "SA0L1E": "Core CPI (excludes food & energy)",
    "SAF": "Food and beverages",
    "SAH": "Housing",
    "SAM": "Medical care",
}

series_ids = [base_series_id + region + item for base_series_id in base_series_ids for region in regions for item in items]

if hit_api:
    df_cpi = bls.get_data(
        series_ids = series_ids,
        years = years,
    )
    df_cpi = bls.clean_data_cpi_unadjusted(df_cpi, base_series_ids, regions, items)
    if save_output:
        save_data(df=df_cpi, filename='01_inflation_data.csv')
else:
    df_cpi = load_data(filename='01_inflation_data.csv')

df_cpi.head()

/Users/annebode/Documents/selfevidence.github.io/data/apis/bls_api.py:80: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,series_id,year,period,value,data_type,region,item,date
0,CUUR0000SA0,2003,M12,184.3,CPI (Unadjusted),National,All items,2003-12-01
1,CUUR0000SA0,2003,M11,184.5,CPI (Unadjusted),National,All items,2003-11-01
2,CUUR0000SA0,2003,M10,185.0,CPI (Unadjusted),National,All items,2003-10-01
3,CUUR0000SA0,2003,M09,185.2,CPI (Unadjusted),National,All items,2003-09-01
4,CUUR0000SA0,2003,M08,184.6,CPI (Unadjusted),National,All items,2003-08-01


In [37]:
df_cpi.describe()

,value,date
count,2500.000000,2500
mean,223.147084,2004-10-15 20:09:35.999999872
min,101.400000,1984-01-01 00:00:00
25%,152.900000,1994-05-24 06:00:00
50%,208.326000,2004-10-16 12:00:00
75%,257.402250,2015-03-08 18:00:00
max,583.875000,2025-08-01 00:00:00
std,97.202951,NaN


### 2.b CPS Weekly Nominal Earnings

In [38]:
# series_ids obtained from: https://data.bls.gov/PDQWeb/le
series_id_dict = {
    
    # First Decile
    'LEU0252911200': {
        'description': '(unadj)- Usual weekly earnings (first decile), Employed full time, Wage and salary workers',
        'percentile': 10,
        'race': 'All'
    },
    # First Quartile
    'LEU0252911300': {
        'description': '(unadj)- Usual weekly earnings (first quartile), Employed full time, Wage and salary workers',
        'percentile': 25,
        'race': 'All'
    },
    # Second Quartile
    'LEU0252881500': {
        'description': '(unadj)- Usual weekly earnings (second quartile), Employed full time, Wage and salary workers',
        'percentile': 50,
        'race': 'All'
    },
    # Third Quartile
    'LEU0252911400': {
        'description': '(unadj)- Usual weekly earnings (third quartile), Employed full time, Wage and salary workers',
        'percentile': 75,
        'race': 'All'
    },
    # Ninth Decile
    'LEU0252911500': {
        'description': '(unadj)- Usual weekly earnings (ninth decile), Employed full time, Wage and salary workers',
        'percentile': 90,
        'race': 'All'
    },
}

if hit_api:
    df_wages = bls.get_data(
        series_ids = list(series_id_dict.keys()),
        years = years,
    )
    df_wages = bls.clean_data_weekly_nominal_earnings(df_wages, series_id_dict)
    if save_output:
        save_data(df=df_wages, filename='01_wage_data.csv')

else:
    df_wages = load_data(filename='01_wage_data.csv')

df_wages.head()

/Users/annebode/Documents/selfevidence.github.io/data/apis/bls_api.py:80: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,series_id,year,period,value,data_type,description,percentile,race,date
0,LEU0252911200,2003,Q04,303.0,CPS Weekly Nominal Earnings,"(unadj)- Usual weekly earnings (first decile),...",10,All,2003-10-01
1,LEU0252911200,2003,Q03,300.0,CPS Weekly Nominal Earnings,"(unadj)- Usual weekly earnings (first decile),...",10,All,2003-07-01
2,LEU0252911200,2003,Q02,301.0,CPS Weekly Nominal Earnings,"(unadj)- Usual weekly earnings (first decile),...",10,All,2003-04-01
3,LEU0252911200,2003,Q01,300.0,CPS Weekly Nominal Earnings,"(unadj)- Usual weekly earnings (first decile),...",10,All,2003-01-01
4,LEU0252911200,2002,Q04,298.0,CPS Weekly Nominal Earnings,"(unadj)- Usual weekly earnings (first decile),...",10,All,2002-10-01


In [39]:
df_wages.describe()

,year,value,percentile,date
count,574.000000,574.000000,574.000000,574
mean,2009.940767,922.968641,50.000000,2010-04-23 20:26:45.574913024
min,1984.000000,278.000000,10.000000,1984-01-01 00:00:00
25%,2003.250000,456.000000,25.000000,2003-10-24 00:00:00
50%,2011.000000,705.500000,50.000000,2011-01-01 00:00:00
75%,2018.000000,1253.000000,75.000000,2018-04-01 00:00:00
max,2025.000000,2905.000000,90.000000,2025-04-01 00:00:00
std,9.663806,602.409176,28.145099,NaN


## 3. Analyze Data

### 3.a Median Wage vs. Inflation (1984 - Present Day)

In [40]:
# normalize wage data
df_1_wage = df_wages[df_wages['percentile'] == 50].copy(deep=True)

first_date = df_1_wage['date'].min()
normalization_wage = df_1_wage[df_1_wage['date'] == first_date]['value'].item()
df_1_wage['wage_adj'] = df_1_wage['value'] / normalization_wage * 100

df_1_wage = df_1_wage.sort_values(by='date')
df_1_wage['date_str'] = pd.to_datetime(df_1_wage['date']).dt.strftime('%Y-%m-%d')

df_1_wage.head()


,series_id,year,period,value,data_type,description,percentile,race,date,wage_adj,date_str
111,LEU0252881500,1984,Q01,323.0,CPS Weekly Nominal Earnings,(unadj)- Usual weekly earnings (second quartil...,50,All,1984-01-01,100.000000,1984-01-01
110,LEU0252881500,1984,Q02,323.0,CPS Weekly Nominal Earnings,(unadj)- Usual weekly earnings (second quartil...,50,All,1984-04-01,100.000000,1984-04-01
109,LEU0252881500,1984,Q03,322.0,CPS Weekly Nominal Earnings,(unadj)- Usual weekly earnings (second quartil...,50,All,1984-07-01,99.690402,1984-07-01
108,LEU0252881500,1984,Q04,335.0,CPS Weekly Nominal Earnings,(unadj)- Usual weekly earnings (second quartil...,50,All,1984-10-01,103.715170,1984-10-01
107,LEU0252881500,1985,Q01,336.0,CPS Weekly Nominal Earnings,(unadj)- Usual weekly earnings (second quartil...,50,All,1985-01-01,104.024768,1985-01-01


In [41]:
# normalize inflation data
first_date = df_cpi['date'].min()
df_normalization_inflation = df_cpi[df_cpi['date']==first_date][['series_id', 'value']].reset_index(drop=True)
df_normalization_inflation.rename(columns={'value': 'baseline_cpi'}, inplace=True)

df_1_inflation = pd.merge(df_cpi, df_normalization_inflation, on = 'series_id', how = 'left')
df_1_inflation['inflation_adj'] = df_1_inflation['value'] / df_1_inflation['baseline_cpi'] * 100
df_1_inflation = df_1_inflation.sort_values(by='date').reset_index(drop=True)

df_1_inflation['date_str'] = pd.to_datetime(df_1_inflation['date']).dt.strftime('%Y-%m-%d')

df_1_inflation.head()

,series_id,year,period,value,data_type,region,item,date,baseline_cpi,inflation_adj,date_str
0,CUUR0000SAH,1984,M01,101.4,CPI (Unadjusted),National,Housing,1984-01-01,101.4,100.0,1984-01-01
1,CUUR0000SAM,1984,M01,104.0,CPI (Unadjusted),National,Medical care,1984-01-01,104.0,100.0,1984-01-01
2,CUUR0000SA0,1984,M01,101.9,CPI (Unadjusted),National,All items,1984-01-01,101.9,100.0,1984-01-01
3,CUUR0000SAF,1984,M01,102.0,CPI (Unadjusted),National,Food and beverages,1984-01-01,102.0,100.0,1984-01-01
4,CUUR0000SA0L1E,1984,M01,102.3,CPI (Unadjusted),National,Core CPI (excludes food & energy),1984-01-01,102.3,100.0,1984-01-01


In [42]:
fig = px.line(df_1_inflation, x='date_str', y='inflation_adj', color='item')

fig.update_layout(

    template='plotly_white',
    width=None,
    height=700,
    autosize=True,
    margin=dict(l=60, r=40, t=100, b=80),

    title={
        'text': '📈 <b>The Rising Cost of Living</b><br><span style="font-size:16px; color:#6b7280">Consumer Price Index by Category (Inflation-Adjusted)</span>',
        'x': 0.5,
        'font': {'size': 20}
    },

    xaxis=dict(
        title={'text': '', 'font': {'size': 16, 'color': '#2E4057'}},
    ),

    yaxis=dict(
        title={'text': 'Inflation-Adjusted CPI', 'font': {'size': 16, 'color': '#2E4057'}},
    ),

    # Legend - moved to top with custom title
    legend=dict(
        title={'text': 'Category', 'font': {'size': 14, 'color': '#2E4057'}},
        orientation='h',
        yanchor='top',
        y=1.0,  # Position below title
        xanchor='center',
        x=0.5,
        font={'size': 12}
    ),
)

fig.update_traces(
    line=dict(width=3),
    hovertemplate='<b>%{fullData.name}</b><br>' +
                  'Date: %{x}<br>' +
                  'CPI: %{y:.0f}<br>' +
                  '<extra></extra>'  # Removes trace box
)

# # Use a curated color palette
# # fig.for_each_trace(lambda t: t.update(line=dict(width=3)))

if save_output:
    save_plotly_figure(
        fig=fig,
        filename='01_cpi_chart_1984',
        for_blog=True
    )

go.FigureWidget(fig)

FigureWidget({
    'data': [{'hovertemplate': '<b>%{fullData.name}</b><br>Date: %{x}<br>CPI: %{y:.0f}<br><extra></extra>',
              'legendgroup': 'Housing',
              'line': {'color': '#636efa', 'dash': 'solid', 'width': 3},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Housing',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '96c984bb-8aba-41cf-bf6e-538bdb19bfc1',
              'x': array(['1984-01-01', '1984-02-01', '1984-03-01', ..., '2025-06-01',
                          '2025-07-01', '2025-08-01'], shape=(500,), dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAWUC9GD3ljh9ZQNZVInQuLF' ... 'GzbHVAepc8GHB2dUClqHYvRod1QA=='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': '<b>%{fullData.name}</b><br>Date: %{x}<br>CPI: %{y:.0f}<br><extra></extra>',
              'legendgroup': 'Medical care',
     

In [46]:
df_plot = pd.merge(
    df_1_inflation[df_1_inflation['item'] == 'Housing'][['date_str', 'inflation_adj']],
    df_1_wage,
    on='date_str',
    how='inner'
)

fig = px.line(df_plot, x='date_str', y=['inflation_adj', 'wage_adj'])

# df_plot_wage = df_1_wage.copy(deep=True)
# fig = px.line(df_plot, x='date_str', y='inflation_adj', color='black')

# fig.update_layout(

#     template='plotly_white',
#     width=None,
#     height=700,
#     autosize=True,
#     margin=dict(l=60, r=40, t=100, b=80),

#     title={
#         'text': '📈 <b>The Rising Cost of Living</b><br><span style="font-size:16px; color:#6b7280">Consumer Price Index by Category (Inflation-Adjusted)</span>',
#         'x': 0.5,
#         'font': {'size': 20}
#     },

#     xaxis=dict(
#         title={'text': '', 'font': {'size': 16, 'color': '#2E4057'}},
#     ),

#     yaxis=dict(
#         title={'text': 'Inflation-Adjusted CPI', 'font': {'size': 16, 'color': '#2E4057'}},
#     ),

#     # Legend - moved to top with custom title
#     legend=dict(
#         title={'text': 'Category', 'font': {'size': 14, 'color': '#2E4057'}},
#         orientation='h',
#         yanchor='top',
#         y=1.0,  # Position below title
#         xanchor='center',
#         x=0.5,
#         font={'size': 12}
#     ),
    
# )

# fig.update_traces(
#     line=dict(width=3),
#     hovertemplate='<b>%{fullData.name}</b><br>' +
#                   'Date: %{x}<br>' +
#                   'CPI: %{y:.0f}<br>' +
#                   '<extra></extra>'  # Removes trace box
# )

# # # Use a curated color palette
# # # fig.for_each_trace(lambda t: t.update(line=dict(width=3)))

go.FigureWidget(fig)

FigureWidget({
    'data': [{'hovertemplate': 'variable=inflation_adj<br>date_str=%{x}<br>_value=%{y}<extra></extra>',
              'legendgroup': 'inflation_adj',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'inflation_adj',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'uid': '42294fbc-cb8a-40f8-9c98-095f2a6a7e49',
              'x': array(['1984-01-01', '1984-04-01', '1984-07-01', '1984-10-01', '1985-01-01',
                          '1985-04-01', '1985-07-01', '1985-10-01', '1986-01-01', '1986-04-01',
                          '1986-07-01', '1986-10-01', '1987-01-01', '1987-04-01', '1987-07-01',
                          '1987-10-01', '1988-01-01', '1988-04-01', '1988-07-01', '1988-10-01',
                          '1989-01-01', '1989-04-01', '1989-07-01', '1989-10-01', '1990-01-01',
                